# Principles of Computer Vision for AI

**Group Project**

**ARI 2129 - Principles of Computer Vision for AI**

---

## Library Imports

Libraries for computer vision, 3D visualization, and point cloud processing are imported here.

In [58]:
import cv2 # Image processing and computer vision
import imageio.v3 as iio # Reading image data
import matplotlib.pyplot as plt # Data visualization and plotting
import numpy as np # Numerical computations and array operations
import time # Performance measurement
import open3d as o3d # 3D scene visualization
import plyfile # Point cloud exportations
import os # File operations 

from collections import namedtuple, defaultdict
from dataclasses import dataclass

## Frame Extraction

This section introduces functions for frame extraction, index management, and color space conversion to prepare our video frames for feature detection and matching.

In [59]:
MatchResult = namedtuple("MatchResult", ["frame_pair", "kp1", "kp2", "pts1", "pts2", "matches"])
DetectorResults = dict[int, tuple[list, np.ndarray]]
Frame = np.ndarray

def get_next_frame_index(frames, current_frame_index):
    next_frame_index = current_frame_index + 1

    if next_frame_index >= len(frames):
        raise ValueError("Next frame index out of bounds")
    
    return next_frame_index

def convert_frames_to_grayscale(*frames):
    gray_frames = []
    for frame in frames:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frames.append(gray)
    
    return tuple(gray_frames)

def get_frame_indices(video_path:str, num_frames:int) -> list:
    # Count total frames by iterating once
    total_frames = sum(1 for _ in iio.imiter(video_path))

    if num_frames > total_frames:
        raise ValueError(f"Requested {num_frames} frames, but video has only {total_frames} frames.")

    indices = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    return indices.tolist()

def load_specific_frames(video_path: str, selected_indices: list, display_frames=True) -> list:
    frames = []
    selected_set = set(selected_indices)
    collected = 0

    for frame_count, frame in enumerate(iio.imiter(video_path)):
        if frame_count in selected_set:
            frames.append(frame)
            collected += 1

            if display_frames:
                plt.imshow(frame)
                plt.title(f'Frame {frame_count}')
                plt.axis('off')
                plt.show()

    print(f"\nFinished loading selected frames.")
    print(f"Total frames processed: {frame_count + 1}")
    
    return frames

## Feature Detection and Description

### Algorithms

This section introduces various feature detection algorithms for identifying distinctive points in images.

- ORB: Oriented FAST and Rotated BRIEF
- SIFT: Scale-Invariant Feature Transform 
- FAST with SIFT descriptors
- Shi-Tomasi corners with ORB descriptors
- Harris corners with ORB descriptors

In [60]:
def detect_orb(gray):
    orb = cv2.ORB_create(nfeatures=3000) 
    keypoints, descriptors = orb.detectAndCompute(gray, None)
    return keypoints, descriptors

def detect_sift(gray):
    sift = cv2.SIFT_create(nfeatures=1500, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return keypoints, descriptors

def detect_fast(gray):
    fast = cv2.FastFeatureDetector_create(threshold=37, nonmaxSuppression=True)
    keypoints = fast.detect(gray, None)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.compute(gray, keypoints)
    return keypoints, descriptors

def detect_shi_tomasi(gray):
    corners = cv2.goodFeaturesToTrack(gray, maxCorners=3000, qualityLevel=0.01, minDistance=10, blockSize=3)
    keypoints = []
    if corners is not None:
        keypoints = [cv2.KeyPoint(float(x), float(y), 1) for [[x, y]] in corners]
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.compute(gray, keypoints)
    return keypoints, descriptors

def detect_harris(gray, block_size=2, ksize=3, k=0.04, threshold_ratio=0.07): 
    gray_f32 = np.float32(gray)
    dst = cv2.cornerHarris(gray_f32, block_size, ksize, k)
    dst = cv2.dilate(dst, None)
    threshold = threshold_ratio * dst.max()
    corners = np.argwhere(dst > threshold)
    keypoints = [cv2.KeyPoint(float(pt[1]), float(pt[0]), 1) for pt in corners]

    orb = cv2.ORB_create()
    keypoints, descriptors = orb.compute(gray, keypoints)
    return keypoints, descriptors

This method processes video frames through a selected feature detector, tracking the number of keypoints found and computational performance. Each frame is first converted to grayscale before detection and result collection.

In [61]:
def detect_features(frames, detector_func):
    detector_results = {}
    keypoint_counts = []
    computation_times = []

    for idx, frame in enumerate(frames):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        start = time.time()
        kp, des = detector_func(gray)
        end = time.time()

        detector_results[idx] = (kp, des)
        keypoint_counts.append(len(kp))
        computation_times.append(end - start)

    return detector_results, keypoint_counts, computation_times

### Visualization

`display_feature_detector_pairs()` helps analyze feature detection results by assigning a distinct color to each detector for consistent visual representation across different plots and frame comparisons.

In [62]:
# Visualisations 
# Colors for matplotlib plots 
DETECTOR_COLORS_MPL = {
    "ORB": 'green',
    "SIFT": 'blue',
    "FAST": 'red',
    "SHI-TOMASI": 'cyan',
    "Harris": 'magenta'
}

DETECTOR_COLORS = {
    "ORB": (0, 255, 0),          
    "SIFT": (255, 0, 0),         
    "FAST": (0, 0, 255),         
    "SHI-TOMASI": (255, 255, 0), 
    "Harris": (255, 0, 255)      
}

In [63]:
def display_feature_detector_pairs(frames, matches: list[MatchResult], detector_name: str):
    color = DETECTOR_COLORS.get(detector_name, (0, 255, 0))

    for match_result in matches:
        i, j = match_result.frame_pair

        img1 = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        img2 = cv2.cvtColor(frames[j], cv2.COLOR_BGR2GRAY)

        img_kp1 = cv2.drawKeypoints(img1, match_result.kp1, None, color=color)
        img_kp2 = cv2.drawKeypoints(img2, match_result.kp2, None, color=color)

        # Show keypoints on individual images
        fig, axes = plt.subplots(1, 2, figsize=(16, 8))
        axes[0].imshow(img_kp1, cmap='gray')
        axes[0].set_title(f"{detector_name} - Frame {i}, Keypoints: {len(match_result.kp1)}")

        axes[1].imshow(img_kp2, cmap='gray')
        axes[1].set_title(f"{detector_name} - Frame {j}, Keypoints: {len(match_result.kp2)}")

        for ax in axes:
            ax.set_xticks([])
            ax.set_yticks([])

        plt.suptitle(f"{detector_name} Keypoints: Frame {i} ↔ Frame {j}")
        plt.tight_layout()
        plt.show()

Functions to compare feature detectors by displaying detected keypoints side-by-side and plotting their performance metrics - including keypoint counts and computation times across video frames.

In [64]:
def compare_detectors_per_frame_cached(frames, detector_results_dict):
    for idx, frame in enumerate(frames):
        frame_results = {}
        for name, frame_data in detector_results_dict.items():            
            kp = frame_data[idx][0]  # (kp, des)
            color = DETECTOR_COLORS.get(name, (0, 255, 0))
            img_with_kp = cv2.drawKeypoints(frame, kp, None, color)
            frame_results[name] = (img_with_kp, len(kp))

        fig, axes = plt.subplots(2, 3, figsize=(15, 15))
        axes = axes.flatten()
        for ax, (name, (img, count)) in zip(axes, frame_results.items()):
            ax.imshow(img)
            ax.set_title(f"{name} ({count})")
            ax.axis('off')
        for ax in axes[len(frame_results):]:
            ax.axis('off')
        plt.suptitle(f"Detector Comparison - Frame {idx}", fontsize=16)
        plt.tight_layout()
        plt.show()

def plot_keypoint_counts_and_comp_time(results):
    # Plots keypoint counts, computation times.
    detector_names = list(results.keys())
    detector_names.remove('frame_indices')
    frame_indices = results['frame_indices']
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 18), sharex=True)
    
    # Plot Keypoint Counts
    for name in detector_names:
        ax1.plot(frame_indices, results[name]['keypoint_counts'], linestyle='-', label=name)
    ax1.set_ylabel('Number of Keypoints')
    ax1.set_title('Keypoint Detection Count per Frame')
    ax1.legend()
    ax1.grid(True)

    # Plot Computation Times
    for name in detector_names:
        ax2.plot(frame_indices, results[name]['computation_times'], linestyle='-', label=name)
    ax2.set_ylabel('Computation Time (seconds)')
    ax2.set_title('Computation Time per Frame')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

    # Print averages
    for name, data in results.items():
        if name != 'frame_indices':
            print(f"Average {name.upper()}: {np.mean(data['keypoint_counts']):.1f} keypoints, {np.mean(data['computation_times']):.4f}s")

Function to create video visualizations of feature detection results. Creates labeled video visualizations showing detected features for each algorithm.

In [65]:
def create_feature_detection_montage(frames, detected_features_all, algorithm_name_func_map, output_dir='feature_detection_montage_videos', fps=10):
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    frame_height, frame_width = frames[0].shape[:2]
    video_writers = {}

    # Initialise a video writer for each algorithm
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    for name in algorithm_name_func_map:
        output_path = os.path.join(output_dir, f"{name}.mp4")
        video_writers[name] = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Set how many times to repeat each frame
    repeat_count = 5  # Increase this to hold the frame longer (e.g., 5 = 0.5 seconds at 10 fps)

    for idx, frame in enumerate(frames):
        for name in algorithm_name_func_map:
            if name not in detected_features_all:
                continue

            keypoints, _ = detected_features_all[name][idx]
            color = DETECTOR_COLORS.get(name, (0, 255, 0))
            img_with_kp = cv2.drawKeypoints(frame, keypoints, None, color)

            # Add label to the frame
            cv2.putText(img_with_kp, f"{name} - Frame {idx + 1}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # Repeat writing this frame
            for _ in range(repeat_count):
                video_writers[name].write(img_with_kp)

        print(f"Processed frame {idx + 1}/{len(frames)}")

    # Release writers
    for writer in video_writers.values():
        writer.release()

    print(f"Videos saved to: {output_dir}")

## Feature Matching and Outlier Rejection

Implementation of feature matching strategies based on descriptor types. FLANN-based matching for float descriptors (SIFT, SURF) and Brute Force matching for binary descriptors (ORB), with Lowe's ratio test to filter unreliable matches.

In [66]:
def get_matcher(detector_type: str):
    detector_type = detector_type.lower()
    if detector_type in ['sift', 'surf', 'fast']:
        # FLANN for float descriptors
        index_params = dict(algorithm=1, trees=5)  # KDTree
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        descriptor_type = 'float'
    elif detector_type in ['orb', 'brief', 'shi-tomasi', 'harris']:
        # Brute-force for binary descriptors
        matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
        descriptor_type = 'binary'
    else:
        raise ValueError(f"Unsupported detector type: {detector_type}")
    
    return matcher, descriptor_type

def match_features(frames, detector_type:str, detector_results:DetectorResults, ratio_thresh=0.75) -> list[MatchResult]:
    matcher, descriptor_type = get_matcher(detector_type)
    match_results = []

    for start_idx in range(len(frames)-1):
        end_idx = start_idx + 1        

        kp1, des1 = detector_results.get(start_idx, (None, None))
        kp2, des2 = detector_results.get(end_idx, (None, None))


        if des1 is None or des2 is None or len(kp1) == 0 or len(kp2) == 0:
            continue
        
        if descriptor_type == 'float':
            des1 = des1.astype(np.float32)
            des2 = des2.astype(np.float32)

        knn_matches = matcher.knnMatch(des1, des2, k=2)
        
        good_matches = []
        for match_pair in knn_matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < ratio_thresh * n.distance:
                    good_matches.append(m)

        if len(good_matches) < 10:
            continue

        pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
        pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])

        result = MatchResult(
            frame_pair=(start_idx, end_idx),
            kp1=kp1,
            kp2=kp2,
            pts1=pts1,
            pts2=pts2,
            matches=good_matches
        )

        match_results.append(result)
        print(f"[{detector_type.upper()}] Matched {start_idx} → {end_idx}: Before Lowe:{len(knn_matches)} matches, After Lowe {len(good_matches)} matches")

    return match_results


In [67]:
# RANSAC removes outlier matches to improve geometric consistency
# Higher threshold = more lenient (accepts points further from predicted position)
def apply_ransac_filter(match_result:MatchResult, reproj_thresh=4.0)-> MatchResult:
    H, mask = cv2.findHomography(match_result.pts1, match_result.pts2, cv2.RANSAC, reproj_thresh)
    mask = mask.ravel()

    inlier_matches = [m for m, keep in zip(match_result.matches, mask) if keep]
    pts1_inliers = match_result.pts1[mask == 1]
    pts2_inliers = match_result.pts2[mask == 1]

    return MatchResult(
        frame_pair=match_result.frame_pair,
        kp1=match_result.kp1,
        kp2=match_result.kp2,
        pts1=pts1_inliers,
        pts2=pts2_inliers,
        matches=inlier_matches
    )

def plot_visual_image_before_after_ransac(img1:Frame, img2:Frame, match_before: MatchResult, match_after: MatchResult, detector_type:str) -> None:
    fig, axes = plt.subplots(1, 2, figsize=(24, 12))

    # Before RANSAC
    img_before = cv2.drawMatches(
        img1, match_before.kp1,
        img2, match_before.kp2,
        match_before.matches,
        None,
        flags=cv2.DrawMatchesFlags_DEFAULT
    )
    axes[0].imshow(img_before)
    axes[0].set_title(f"Lowe's Ratio Before RANSAC - Matches: {len(match_before.matches)}", fontsize=20)
    axes[0].axis('off')

    # After RANSAC
    img_after = cv2.drawMatches(
        img1, match_after.kp1,
        img2, match_after.kp2,
        match_after.matches,
        None,
        flags=cv2.DrawMatchesFlags_DEFAULT
    )
    axes[1].imshow(img_after)
    axes[1].set_title(f"After RANSAC - Inliers: {len(match_after.matches)}", fontsize=20)
    axes[1].axis('off')

    fig.suptitle(f"{detector_type} Frames:{match_before.frame_pair}", fontsize=20)
    plt.tight_layout()
    plt.show()
    

def plot_before_after_ransac_detected_features_all_algos(feature_matching_results_all_algos):
    plt.figure(figsize=(12, 6))

    for detector_type, result_dict in feature_matching_results_all_algos.items():
        matches_before = result_dict['matches']
        matches_after = result_dict['ransac']

        total_inliers = sum(len(result_after.matches) for result_after in matches_after)
        total_matches = sum(len(result_before.matches) for result_before in matches_before)
        if total_matches == 0:
            print(f"Skipping {detector_type}: no matches")
            continue

        inlier_ratio_overall = total_inliers / total_matches

        # Compute per-frame-pair inlier ratios
        ratios = [
            len(after.matches) / len(before.matches)
            if len(before.matches) > 0 else 0
            for before, after in zip(matches_before, matches_after)
        ]

        color = DETECTOR_COLORS_MPL.get(detector_type, 'green')  # Fallback to black if undefined
        label = f"{detector_type} (overall: {inlier_ratio_overall:.2f})"
        plt.plot(ratios, label=label, color=color)

    plt.title("Inlier Ratio per Frame Pair (RANSAC Filtered)")
    plt.xlabel("Frame Pair Index")
    plt.ylabel("Inlier Ratio (After / Before RANSAC)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

## Essential/Fundamental Matrix Computation

Methods for computing and validating the geometric relationship between image pairs. The fundamental matrix captures this relationship, while epipolar line visualization and error metrics help verify the accuracy of the computed geometry.

In [68]:
# Compute the fundamental matrix from matched keypoints
def compute_fundamental_matrix(matches, keypoints1, keypoints2, method = cv2.FM_RANSAC, ransac_threshold = 3.0, confidence = 0.99):
  # Extract coordinates of matched keypoints
  points1 = np.float32([keypoints1[m.queryIdx].pt for m in matches])
  points2 = np.float32([keypoints2[m.trainIdx].pt for m in matches])
  
  # Compute the fundamental matrix
  fundamental_matrix, inlier_mask = cv2.findFundamentalMat(points1, points2, method = method, ransacReprojThreshold = ransac_threshold, confidence = confidence)
  
  # Convert mask to binary array for easier filtering
  if fundamental_matrix is None or fundamental_matrix.shape != (3,3):
    raise ValueError("Failed to compute a valid fundamental matrix")
  
  return fundamental_matrix, inlier_mask

# Clip epipolar lines to image boundaries
def draw_epipolar_line_clipped(img_shape, line):
  h, w = img_shape[:2]
    
  # Find intersections with image borders
  points = []
    
  # Left border (x = 0)
  if abs(line[1]) > 1e-6:
    y = -line[2] / line[1]
    if 0 <= y <= h:
      points.append((0, int(y)))
    
  # Right border (x = w-1)
  if abs(line[1]) > 1e-6:
    y = -(line[2] + line[0] * (w-1)) / line[1]
    if 0 <= y <= h:
      points.append((w-1, int(y)))
    
  # Top border (y = 0)
  if abs(line[0]) > 1e-6:
    x = -line[2] / line[0]
    if 0 <= x <= w:
      points.append((int(x), 0))
    
  # Bottom border (y = h-1)
  if abs(line[0]) > 1e-6:
    x = -(line[2] + line[1] * (h-1)) / line[0]
    if 0 <= x <= w:
      points.append((int(x), h-1))
    
  # Remove duplicate points and return first two
  unique_points = []
  for p in points:
    if not any(abs(p[0]-up[0]) < 2 and abs(p[1]-up[1]) < 2 for up in unique_points):
      unique_points.append(p)
    
  return unique_points[:2] if len(unique_points) >= 2 else []

# Visualise epipolar lines to verify the fundamental matrix
def visualise_epipolar_lines(img1, img2, points1, points2, fundamental_matrix, sample_size = 20):
  # Sample points if there are too many
  if len(points1) > sample_size:
    indices = np.random.choice(len(points1), sample_size, replace = False)
    pts1 = points1[indices]
    pts2 = points2[indices]
  else:
    pts1 = points1
    pts2 = points2

  # Create a new figure for each call
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 8))

  # Convert images to RGB for matplotlib
  img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
  img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

  # Display the first image
  ax1.imshow(img1_rgb) # Use RGB image
  ax1.set_title('Epipolar Lines on Image 1')
  ax1.axis('off')

  # Display the second image
  ax2.imshow(img2_rgb) # Use RGB image
  ax2.set_title('Epipolar Lines on Image 2')
  ax2.axis('off')

  # Draw epipolar lines on both images
  for i in range(len(pts1)):
    # Draw points
    ax1.plot(pts1[i, 0], pts1[i, 1], 'ro', markersize = 6)
    ax2.plot(pts2[i, 0], pts2[i, 1], 'ro', markersize = 6)

    # Compute epipolar line in second image for point in first image
    line2 = cv2.computeCorrespondEpilines(pts1[i].reshape(-1, 1, 2), 1, fundamental_matrix)
    line2 = line2.reshape(-1)

    # Get clipped line endpoints for image 2
    endpoints2 = draw_epipolar_line_clipped(img2.shape, line2)
    if len(endpoints2) == 2:
        ax2.plot([endpoints2[0][0], endpoints2[1][0]], [endpoints2[0][1], endpoints2[1][1]], 'g-', linewidth=1)

    # Compute epipolar line in first image for point in second image
    line1 = cv2.computeCorrespondEpilines(pts2[i].reshape(-1, 1, 2), 2, fundamental_matrix)
    line1 = line1.reshape(-1)

    # Get clipped line endpoints for image 1
    endpoints1 = draw_epipolar_line_clipped(img1.shape, line1)
    if len(endpoints1) == 2:
        ax1.plot([endpoints1[0][0], endpoints1[1][0]], [endpoints1[0][1], endpoints1[1][1]], 'g-', linewidth=1)

  plt.tight_layout()
  plt.show()
  plt.close(fig)

# Calculate the epipolar geometry error to evaluate the quality of the fundamental matrix
def epipolar_error(points1, points2, fundamental_matrix):
  # Convert each point to homogeneous coordinates
  homogeneous_points1 = np.hstack((points1, np.ones((points1.shape[0],1))))
  homogeneous_points2 = np.hstack((points2, np.ones((points2.shape[0],1))))
  
  # Calculate epipolar lines for points in image 1
  lines2 = np.dot(homogeneous_points1, fundamental_matrix.T)
  # Normalise lines
  norms2 = np.sqrt(lines2[:, 0]**2 + lines2[:, 1]**2)
  lines2 = lines2 / norms2.reshape(-1,1)
  # Calculate the distance from points in image 2 to their corresponding epipolar lines
  dist2 = np.abs(np.sum(lines2 * homogeneous_points2, axis = 1))
  
  # Calculate epipolar lines for points in image 2
  lines1 = np.dot(homogeneous_points2, fundamental_matrix)
  # Normalise lines
  norms1 = np.sqrt(lines1[:, 0]**2 + lines1[:, 1]**2)
  lines1 = lines1 / norms1.reshape(-1,1)
  # Calculate the distance from points in image 1 to their corresponding epipolar lines
  dist1 = np.abs(np.sum(lines1 * homogeneous_points1, axis = 1))
  
  metrics = {
    "mean_error": (np.mean(dist1) + np.mean(dist2)) / 2,
    "max_error": max(np.max(dist1), np.max(dist2)),
    "std_error": (np.std(dist1) + np.std(dist2)) / 2
  }
  
  return metrics

# Main function to compute the fundamental matrix
def process_fundamental_matrix(imgs, matches, keypoints1, keypoints2, visualise = True):
  # Compute fundamental matrix
  print("Computing fundamental matrix...")
  F, inlier_mask = compute_fundamental_matrix(matches, keypoints1, keypoints2)
    
  # Filter matches based on inlier mask
  inlier_matches = [m for i, m in enumerate(matches) if inlier_mask[i]]
  print(f"Inlier matches: {len(inlier_matches)} out of {len(matches)} ({len(inlier_matches) / len(matches) * 100:.2f}%)")
    
  # Extract coordinates of inlier keypoints
  inlier_points1 = np.float32([keypoints1[m.queryIdx].pt for m in inlier_matches])
  inlier_points2 = np.float32([keypoints2[m.trainIdx].pt for m in inlier_matches])
    
  # Calculate error metrics
  error_metrics = epipolar_error(inlier_points1, inlier_points2, F)
  print(f"Mean epipolar error: {error_metrics['mean_error']:.4f} pixels")
  print(f"Max epipolar error: {error_metrics['max_error']:.4f} pixels")
  print(f"Std. epipolar error: {error_metrics['std_error']:.4f} pixels")
        
  # Visualise epipolar lines if requested
  if visualise and len(imgs) >= 2:
      visualise_epipolar_lines(imgs[0], imgs[1], inlier_points1, inlier_points2, F)
    
  # Prepare results
  results = {
      "fundamental_matrix": F,
      "inlier_mask": inlier_mask,
      "inlier_matches": inlier_matches,
      "inlier_points1": inlier_points1,
      "inlier_points2": inlier_points2,
      "error_metrics": error_metrics,
  }
    
  return results

## Camera Pose Estimation

Implementation of camera pose recovery from matched features. Starting with approximate camera intrinsics, the pipeline converts fundamental to essential matrix, recovers relative camera positions, and validates the estimated poses through geometric constraints and visualization.

In [69]:
@dataclass
class CameraIntrinsics:
  fx: float
  fy: float
  cx: float
  cy: float
  
  def matrix(self):
    return np.array([
      [self.fx, 0, self.cx],
      [0, self.fy, self.cy],
      [0, 0, 1]
    ],dtype = np.float64)
    
# Estimate approximate intrinsics for a phone camera
def estimate_camera_intrinsics(image_shape):
  height, width = image_shape
  
  fov_degrees = 75
  fov_radians = np.radians(fov_degrees)
  f = (width / 2.0) / np.tan(fov_radians / 2.0)
  
  # Principal point at image center
  cx = width / 2
  cy = height / 2
  
  print(f"Intrinsics fx = fy = {f:.1f}, cx = {cx:.1f}, cy = {cy:.1f}")
  return CameraIntrinsics(f, f, cx, cy)

# Compute the essential matrix from the fundamental matrix using the formula:
# E = K^T * F * K
def convert_fundamental_to_essential_matrix(F, K):
  E = K.T @ F @ K
  return E

# Recover relative camera pose (R, t) from the essential matrix
def recover_camera_pose(E, pts1, pts2, K):
  pts1 = np.asarray(pts1, dtype = np.float32)
  pts2 = np.asarray(pts2, dtype = np.float32)
  
  # Make sure points have correct shape
  if pts1.ndim == 1:
    pts1 = pts1.reshape(-1,2)
  if pts2.ndim == 1:
    pts2 = pts2.reshape(-1,2)
  
  try:
    # Chirality problem - Funtion automatically tests all 4 possible combinations of R and t
    _, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)
    
    if mask is not None:
      actual_inliers = np.sum(mask > 0)
    else:
      actual_inliers = 0
      
    print(f"Pose Inliers: {actual_inliers}/{len(pts1)}")
    
    if actual_inliers < 5:
      print("Very few inliers for pose recovery")
      
    return R, t, mask, actual_inliers
  except cv2.error as e:
    print(f"OpenCV error in pose recovery: {e}")
     # Return identity transformation as fallback
    return np.eye(3), np.zeros((3, 1)), np.zeros(len(pts1), dtype = np.uint8), 0

# Validity checks on the recovered pose
def validate_pose(R, t):
  metrics = {}
  det_R = np.linalg.det(R)
  metrics['det_R'] = det_R
  metrics['is_rotation_valid'] = np.abs(det_R - 1.0) < 1e-6

  orthogonality_error = np.linalg.norm(R @ R.T - np.eye(3))
  metrics['orthogonality_error'] = orthogonality_error
  metrics['is_orthogonal'] = orthogonality_error < 1e-6

  t_mag = np.linalg.norm(t)
  metrics['translation_magnitude'] = t_mag

  trace = np.trace(R)
  trace = np.clip(trace, -1.0, 3.0)
  angle_rad = np.arccos((trace - 1) / 2)
  metrics['rotation_angle_deg'] = np.degrees(angle_rad)

  return metrics

# Simple 3D plot of the initial and recovered camera pose
def visualise_camera_poses(R, t):
  try:
    fig = plt.figure(figsize = (10,8))
    ax = fig.add_subplot(111, projection = '3d')

    cam1 = np.array([0, 0, 0])
    cam2 = -R.T @ t.flatten()  # Camera center in world coordinates
    
    if cam2.ndim > 1:
      cam2 = cam2.flatten()
    
    ax.scatter(cam1[0], cam1[1], cam1[2], c = 'blue', s = 50, label = 'Camera 1 (origin)')
    ax.scatter(cam2[0], cam2[1], cam2[2], c = 'red', s = 50, label = 'Camera 2')
    ax.plot([cam1[0], cam2[0]], [cam1[1], cam2[1]], [cam1[2], cam2[2]], 'k--', linewidth = 2, label = 'Baseline')

    # Add coordinate axes for camera orientations
    axes_length = 0.3
        
    # Camera 1 axes (identity rotation)
    ax.plot([0, axes_length], [0, 0], [0, 0], 'r-', alpha = 0.7, linewidth = 2)  # X-axis
    ax.plot([0, 0], [0, axes_length], [0, 0], 'g-', alpha = 0.7, linewidth = 2)  # Y-axis
    ax.plot([0, 0], [0, 0], [0, axes_length], 'b-', alpha = 0.7, linewidth = 2)  # Z-axis
        
    # Camera 2 axes (rotated)
    axes = axes_length * R.T  # Camera coordinate axes in world frame
    origin = cam2
        
    x_end = origin + axes[:, 0]
    y_end = origin + axes[:, 1] 
    z_end = origin + axes[:, 2]
        
    ax.plot([origin[0], x_end[0]], [origin[1], x_end[1]], [origin[2], x_end[2]], 'r-', alpha = 0.7, linewidth = 2)
    ax.plot([origin[0], y_end[0]], [origin[1], y_end[1]], [origin[2], y_end[2]], 'g-', alpha = 0.7, linewidth = 2)
    ax.plot([origin[0], z_end[0]], [origin[1], z_end[1]], [origin[2], z_end[2]], 'b-', alpha = 0.7, linewidth = 2)

    ax.set_xlabel('X') # red line
    ax.set_ylabel('Y') # green line
    ax.set_zlabel('Z') # blue line
    ax.set_title('Camera Poses')
    ax.legend()
        
    # Set equal aspect ratio
    max_range = max(np.abs(cam2).max(), 0.5)
    ax.set_xlim([-max_range, max_range])
    ax.set_ylim([-max_range, max_range])
    ax.set_zlim([-max_range, max_range])
        
    plt.tight_layout()
    plt.show()
  except Exception as e:
    print(f"Visualisation error: {e}")
    print("Skipping visualisation process")

# Takes in fundamental matrix and inlier points computed from stage 5
def run_pose_estimation(F, pts1, pts2, frame_shape, visualise = True):
  print("Camera Pose Estimation: ")
  
  if F is None or F.shape != (3,3):
    raise ValueError("Invalid fundamental matrix")
  
  if len(pts1) < 5 or len(pts2) < 5:
    raise ValueError("Needs at least 5 point correspondences")
  
  try:
    # Estimate the intrinsics
    intrinsics = estimate_camera_intrinsics(frame_shape)
    K = intrinsics.matrix()
    
    # Deduce the essential matrix
    E = convert_fundamental_to_essential_matrix(F, K)
    print("Essential Matrix\n", E)
    
    # Check if essential matrix is valid using SVD
    U, S, Vt = np.linalg.svd(E)
    print(f"Essential matrix singular values: {S}")
    
    # Recover pose
    R, t, mask, actual_inliers = recover_camera_pose(E, pts1, pts2, K)
    
    # Validation step
    pose_metrics = validate_pose(R, t)
    print("\nPose Validation: ")
    for k, v in pose_metrics.items():
      print(f"{k}: {v}")
      
    if not pose_metrics['is_rotation_valid']:
      print("Invalid rotation matrix")
      
    if pose_metrics['rotation_angle_deg'] > 90:
      print(f"Large rotation angle {pose_metrics['rotation_angle_deg']:.1f}")
    
    # Check for degenerate 
    if pose_metrics['rotation_angle_deg'] < 0.1:
      print("Very small rotation - possible degenerate case")
    
    # Visualise the initial and recovered camera pose
    if visualise:
      visualise_camera_poses(R, t)
      
    return {
      "R": R,
      "t": t,
      "K": K,
      "E": E,
      "mask": mask,
      "metrics": pose_metrics,
      "actual_inliers": actual_inliers
    }
  except Exception as e:
    print(f"Error in pose estimation: {e}")
    return {"error": str(e)}

In [70]:
# Wrapper to integrate stages 4-6 together
def run_pose_estimation_from_matches(match_result, frame_shape, fundamental_matrix, visualise):
  return run_pose_estimation(F = fundamental_matrix, pts1 = match_result.pts1, pts2 = match_result.pts2, frame_shape = frame_shape, visualise=visualise)

## 3D Point Triangulation and Scene Visualisation

Methods for reconstructing 3D scenes from matched features and camera poses. The pipeline includes point triangulation, color mapping, bundle adjustment for pose refinement, quality analysis, and tools for visualizing and saving the reconstructed point cloud.

In [71]:
def triangulate_points(K, camera1_pose, camera2_pose, points1, points2):
    # Triangulate points in 3D space from 2D correspondences and camera parameters

    # Input validation
    if points1.shape[0] != points2.shape[0]:
        print("Error: Input point arrays must have the same number of points.")
        return np.array([])

    # Deconstruct poses
    R1, t1 = camera1_pose
    R2, t2 = camera2_pose

    # Compute projection matrices P = K[R|t]
    P1 = K @ np.hstack((R1, t1))
    P2 = K @ np.hstack((R2, t2))
        
    # Triangulate points using OpenCV
    # cv2.triangulatePoints requires points in (2, N) format, so we transpose our (N, 2) inputs
    points_4d_hom = cv2.triangulatePoints(P1, P2, points1.T, points2.T)
    
    # Convert homogeneous coordinates (x,y,z,w) to Euclidean coordinates (x,y,z)
    points_3d = (points_4d_hom[:3, :] / points_4d_hom[3, :]).T
    
    return points_3d

def get_pixel_colors(img, coordinate_list):
    # Initialize empty array for color values
    pixel_values = []
    image_height, image_width = img.shape[:2]
    
    # Process each coordinate
    for coord in coordinate_list:
        col, row = map(lambda x: int(x + 0.5), coord)
        
        # Validate coordinates
        is_valid = (0 <= col < image_width) and (0 <= row < image_height)
        
        if is_valid:
            # Normalize RGB values to [0,1] range
            rgb = img[row, col].astype(float) / 255
        else:
            rgb = np.array([0.5, 0.5, 0.5])  # Gray color for invalid points
            
        pixel_values.append(rgb)
    
    return np.array(pixel_values)

def visualize_reconstruction(points_3d, colors, window_name):
    # Create a point cloud object
    pcd = o3d.geometry.PointCloud()

    # Flip Y and Z coordinates for correct orientation
    points_3d = points_3d.copy()
    points_3d[:, 1] = -points_3d[:, 1] # Y coordinate
    points_3d[:, 2] = -points_3d[:, 2] # Z coordinate

    pcd.points = o3d.utility.Vector3dVector(points_3d)
    # Add RGB colors to the points
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    print("\nVisualizing point cloud.")
    print("Controls: Mouse drag=rotate, Shift+drag=pan, Scroll=zoom, Ctrl+drag=Translate")
    print("Press 'q' to close the window.")

    # Display the 3D point cloud
    o3d.visualization.draw_geometries([pcd], window_name=window_name)

def bundle_adjust_poses(camera_poses, all_points_3d, all_matches, K, max_iterations=10):
    # Iterative bundle adjustment to refine camera poses and 3D points.
    print(f"Starting bundle adjustment with {len(camera_poses)} poses...")
    
    # Return early if no 3D points exist
    if not all_points_3d:
        print("No 3D points provided for bundle adjustment!")
        return camera_poses, []
    
    # Create copies to avoid modifying inputs
    refined_poses = camera_poses.copy()
    refined_points_3d = [points.copy() for points in all_points_3d]  # Keep as list of arrays
    
    # Iteratively refine poses up to max_iterations times
    for iter_idx in range(max_iterations):
        total_reproj_error = 0
        pose_updates = 0
        
        # Iterate through each camera pose after first reference camera
        for cam_idx in range(1, len(camera_poses)):
            # Skip if we don't have matching data for this camera
            if cam_idx - 1 >= len(all_matches):
                continue
                
            # Get corresponding 2D-3D point pairs for this camera
            match_data = all_matches[cam_idx - 1]
            frame_points_3d = refined_points_3d[cam_idx - 1]  # Use refined points
            image_points = match_data.pts2.astype(np.float32)
            object_points = frame_points_3d.astype(np.float32)
            
            # Need at least 6 points for reliable pose estimation
            if len(object_points) < 6:
                continue
                
            try:
                # Initialize with current pose estimate
                R_current, t_current = refined_poses[cam_idx]
                rvec_init, _ = cv2.Rodrigues(R_current)
                tvec_init = t_current.flatten()
                
                # Use PnP RANSAC to optimize the camera pose
                success, rvec_refined, tvec_refined, inliers = cv2.solvePnPRansac(
                    object_points.reshape(-1, 1, 3),
                    image_points.reshape(-1, 1, 2),
                    K,
                    None,
                    rvec_init,
                    tvec_init,
                    useExtrinsicGuess=True,
                    iterationsCount=100,
                    reprojectionError=2.0,
                    confidence=0.99,
                    flags=cv2.SOLVEPNP_ITERATIVE
                )
                
                # Only update pose if we have a good solution with sufficient inliers
                if success and inliers is not None and len(inliers) > len(object_points) * 0.5:
                    R_refined, _ = cv2.Rodrigues(rvec_refined)
                    t_refined = tvec_refined.reshape(3, 1)
                    
                    refined_poses[cam_idx] = (R_refined, t_refined)
                    pose_updates += 1
                    
            except cv2.error as e:
                print(f"Camera {cam_idx}: PnP failed - {e}")
                continue
                
        # Early stopping if very few poses were updated
        if pose_updates < len(camera_poses) * 0.2:
            print("Early stopping due to few pose updates")
            break
    
    return refined_poses, refined_points_3d

def analyze_reconstruction_quality(points_3d, points1, points2, P1, P2, baseline_length):
    # Function to analyze quality metrics for 3D reconstruction

    # Convert 3D points to homogeneous coordinates by adding 1 as fourth dimension
    # Project 3D points back into 2D using both camera matrices
    points_3d_hom = np.hstack([points_3d, np.ones((len(points_3d), 1))])
    reproj1 = (P1 @ points_3d_hom.T)
    reproj2 = (P2 @ points_3d_hom.T)

    # Convert back from homogeneous to euclidean coordinates
    reproj1 = (reproj1[:2, :] / reproj1[2, :]).T
    reproj2 = (reproj2[:2, :] / reproj2[2, :]).T
    
    # Calculate reprojection error as L2 distance between original and reprojected points
    error1 = np.linalg.norm(reproj1 - points1, axis=1)
    error2 = np.linalg.norm(reproj2 - points2, axis=1)
    mean_reproj_error = (np.mean(error1) + np.mean(error2)) / 2
    
    # Calculate distances of reconstructed 3D points from origin
    distances = np.linalg.norm(points_3d, axis=1)
        
    # Print statistics
    print(f"\nReconstruction Quality Metrics:")
    print(f"Points: {len(points_3d)}")
    print(f"Reprojection error: {mean_reproj_error:.2f} pixels")
    print(f"Baseline: {baseline_length:.3f}")
    print(f"Scale ratio: {np.max(distances)/baseline_length:.1f}x")
    
    # Return dictionary with key metrics
    return {
        'mean_reprojection_error': mean_reproj_error,
        'num_points': len(points_3d),
        'scale_ratio': np.max(distances)/baseline_length
    }

def save_point_cloud_ply(filename, points, colors):
    # Save 3D points with colors to a .ply file

    # Create reconstructions directory if it doesn't exist
    reconstruction_dir = 'reconstructions'
    os.makedirs(reconstruction_dir, exist_ok=True)

    # Join path with filename
    filepath = os.path.join(reconstruction_dir, filename)

    # Flip coordinates for correct orientation
    points = points.copy()
    points[:, 1] = -points[:, 1]  # Y coordinate
    points[:, 2] = -points[:, 2]  # Z coordinate

    # Ensure points and colors are float32 / uint8
    points = points.astype(np.float32)
    colors = (colors * 255).astype(np.uint8)  # If colors were normalized [0,1]

    # Create a structured array
    vertex_data = np.array(
        [tuple(p) + tuple(c) for p, c in zip(points, colors)],
        dtype=[
            ('x', 'f4'), ('y', 'f4'), ('z', 'f4'),
            ('red', 'u1'), ('green', 'u1'), ('blue', 'u1')
        ]
    )

    el = plyfile.PlyElement.describe(vertex_data, 'vertex')
    plyfile.PlyData([el]).write(filepath)
    print(f"Saved point cloud to {filepath}")

Main reconstruction pipeline that combines all previous components. Starting with camera poses and matched features, it performs sequential reconstruction by triangulating 3D points, optionally refines results through bundle adjustment, analyzes reconstruction quality, and generates the final point cloud with color information.

In [72]:
def reconstruct_scene(pose_estimations, ransac_results, Frames, detector_name, use_bundle_adjustment=True, visualize=True):
    # Initialize first camera pose
    R_global = np.eye(3)
    t_global = np.zeros((3, 1))
    camera_poses = [(R_global, t_global)] # Store camera poses as list of (R,t) tuples, starting with initial pose

    # Lists to store 3D points and their RGB colors
    all_points_3d = []
    all_colors = []

    # List to store reconstruction quality metrics
    all_quality_metrics = []

    # Get camera intrinsic matrix K from first pose estimation result
    if not pose_estimations:
        print("No pose estimations available to proceed with triangulation.")
        exit()
        
    K = pose_estimations[0]['K']

    # Process each pose estimation sequentially to build up 3D reconstruction
    for i, pose_result in enumerate(pose_estimations):
        # Extract relative pose (R,t) between current frame pair
        R_rel = pose_result.get("R")
        t_rel = pose_result.get("t")

        # Skip if invalid pose
        if R_rel is None or t_rel is None:
            print(f"Skipping pair {ransac_results[i].frame_pair} due to invalid pose.")
            continue

        # Get current RANSAC result
        ransac_res = ransac_results[i]
        
        # Calculate global pose
        R_new_global = R_global @ R_rel
        t_new_global = t_global + R_global @ t_rel
        
        # Initial 3D point triangulation
        points_3d = triangulate_points(
            K,
            (R_global, t_global),
            (R_new_global, t_new_global),
            ransac_res.pts1,
            ransac_res.pts2
        )
        
        # Get RGB color values for each 3D point from first frame
        frame_idx_1, _ = ransac_res.frame_pair
        colors = get_pixel_colors(Frames[frame_idx_1], ransac_res.pts1)

        # Store results for this frame pair
        all_points_3d.append(points_3d)
        all_colors.append(colors)
        
        print(f"Triangulated {len(points_3d)} points.")

        # Update global pose for next iteration
        R_global = R_new_global
        t_global = t_new_global
        camera_poses.append((R_global, t_global))

    print("")

    # Apply bundle adjustment
    if use_bundle_adjustment and len(camera_poses) > 1 and all_points_3d:
        print("\nApplying bundle adjustment...")
        refined_poses, refined_points_3d = bundle_adjust_poses(
            camera_poses, all_points_3d, ransac_results, K
        )
        
        # Use refined results
        camera_poses = refined_poses
        all_points_3d = refined_points_3d
        
        print("Bundle adjustment completed.")

    # Calculate reconstruction quality metrics 
    for i in range(len(ransac_results)):
        if i + 1 >= len(camera_poses):
            continue
            
        ransac_res = ransac_results[i]
        R1, t1 = camera_poses[i]
        R2, t2 = camera_poses[i + 1]
        
        # Use existing points if available, otherwise triangulate
        if i < len(all_points_3d):
            points_3d = all_points_3d[i]
        else:
            points_3d = triangulate_points(
                K, (R1, t1), (R2, t2),
                ransac_res.pts1, ransac_res.pts2
            )
            all_points_3d.append(points_3d)
        
        # Get colors if not already stored
        if i >= len(all_colors):
            colors = get_pixel_colors(Frames[ransac_res.frame_pair[0]], ransac_res.pts1)
            all_colors.append(colors)
        
        # Construct projection matrices for quality analysis
        P1 = K @ np.hstack((R1, t1))
        P2 = K @ np.hstack((R2, t2))
        
        # Analyze quality
        quality = analyze_reconstruction_quality(
            points_3d,
            ransac_res.pts1,
            ransac_res.pts2,
            P1, P2,
            np.linalg.norm(t2 - t1)
        )
        
        all_quality_metrics.append(quality)
        
    # Compute overall reconstruction quality metrics across all frame pairs
    if all_quality_metrics:
        avg_error = np.mean([m['mean_reprojection_error'] for m in all_quality_metrics])
        avg_scale = np.mean([m['scale_ratio'] for m in all_quality_metrics])
        total_points = sum([m['num_points'] for m in all_quality_metrics])

        print(f"\n{detector_name} Final Reconstruction Quality Summary:")
        print(f"Average Reprojection Error: {avg_error:.2f} pixels")
        print(f"Average Scale Ratio: {avg_scale:.1f}x")
        print(f"Total Reconstructed Points: {total_points}")

    # Combine all frame pairs results into final point cloud
    if all_points_3d:
        final_points_3d = np.concatenate(all_points_3d, axis=0)
        final_colors = np.concatenate(all_colors, axis=0)
        
        # Display final combined point cloud
        print(f"\nTotal reconstructed points: {len(final_points_3d)}")
        
        if visualize:
            visualize_reconstruction(final_points_3d, final_colors, window_name=f'{detector_name} 3D Scene Reconstruction')
        save_point_cloud_ply(f"{detector_name}_reconstruction.ply", final_points_3d, final_colors)

    else:
        print("Could not reconstruct any 3D points.")

## Main Driver Code

Setup of input parameters, output control flags, and algorithm mappings for the reconstruction pipeline.

In [73]:
# Input Variables
VIDEO_PATH = "../videos/Video_Selection_CV.mp4"
NUMBER_OF_FRAME_PAIRS = 11

# Output Toggles
PLOT_DETECTOR_BENCHMARK_COMPARISONS = False
PLOT_FEATURE_DETECTOR_PAIRS = False
PLOT_RANSAC_FILTERING = False
PLOT_FUNDAMENTAL_MATRIX_COMPUTATION = False
PLOT_CAMERA_POSE_ESTIMATIONS = False
VISUALIZE_3D_RECONTRUCTION = True
CREATE_FEATURE_MONTAGE_VIDEOS = False

# 3D Scene Reconstruction Toggles
USE_BUNDLE_ADJUSTMENT = True

ALGORITHM_NAME_FUNC_MAP = {"ORB" : detect_orb,
                            "SIFT": detect_sift,
                            "FAST": detect_fast,
                            "SHI-TOMASI": detect_shi_tomasi,
                            "HARRIS": detect_harris}

Initial frame extraction from video using specified indices. This step prepares the base frames that will be processed through the reconstruction pipeline.

In [74]:
# Frame preparation
indices = get_frame_indices(VIDEO_PATH, NUMBER_OF_FRAME_PAIRS)
print(f"Overlapping indices: {indices}")
Frames = load_specific_frames(VIDEO_PATH, selected_indices=indices, display_frames=False)

Overlapping indices: [0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240]

Finished loading selected frames.
Total frames processed: 241


Main processing loop that runs each feature detector through the reconstruction pipeline:
1. Feature detection and timing
2. Feature matching between frames
3. RANSAC-based outlier filtering
4. Fundamental matrix computation
5. Camera pose estimation
6. 3D reconstruction with optional bundle adjustment

In [ ]:
feature_matching_results_all_algos = {}

results_benchmark_data = {
    'frame_indices': list(range(len(Frames)))
}

detected_features_all = {}

# Loop through each detector type and function pair to process feature matching
for detector_type, detector_func in ALGORITHM_NAME_FUNC_MAP.items():
    print(f"\nRunning:{detector_type}, func:{detector_func.__name__}")

    # 1. Detect Features
    detected_features, keypoint_counts, computation_times = detect_features(Frames, detector_func)
    results_benchmark_data[detector_type] = {
        'keypoint_counts': keypoint_counts,
        'computation_times': computation_times
    }
    detected_features_all[detector_type] = detected_features

    # 2. Match Features
    match_results = match_features(Frames, detector_type, detected_features)
    if not match_results: raise RuntimeError("No valid matches found!")
    if PLOT_FEATURE_DETECTOR_PAIRS: display_feature_detector_pairs(Frames, match_results, detector_type)  
    
    # 3. RANSAC Filtering
    ransac_results = [apply_ransac_filter(res, reproj_thresh=16) for res in match_results]
    print(f"Number of match results: {len(match_results)}")
    feature_matching_results_all_algos[detector_type] = {
        "matches": match_results,
        "ransac": ransac_results
    }

    # Visualize matches before and after RANSAC filtering
    if PLOT_RANSAC_FILTERING:
        for before, after in zip(match_results, ransac_results):
                    i, j = before.frame_pair
                    print(f"\nVisualising Matches: Frame Pair ({i}, {j})")
                    plot_visual_image_before_after_ransac(Frames[i], Frames[j], before, after, detector_type)
    

    # 4. Fundamental Matrix and Epipolar Geometry
    print("\nFundamental Matrix and Epipolar Geometry:")
    fundamental_results_list = []

    # Process each RANSAC result to compute fundamental matrix
    for res in ransac_results:
        i, j = res.frame_pair
        print(f"\nFundamental Matrix Computation for Frame Pair ({i}, {j})")
        imgs = [Frames[i], Frames[j]]
        fundamental_result = process_fundamental_matrix(imgs, res.matches, res.kp1, res.kp2, visualise=PLOT_FUNDAMENTAL_MATRIX_COMPUTATION)
        fundamental_results_list.append(fundamental_result)

    # 5. Pose Estimation
    print("\nCamera Pose Estimation:")
    pose_estimations = []
    frame_shape = (Frames[0].shape[0], Frames[0].shape[1])  # Get frame dimensions

    # Process each fundamental matrix result and RANSAC result
    for idx, (fundamental_data, res) in enumerate(zip(fundamental_results_list, ransac_results)):
        print(f"\nPose Estimation for Frame Pair {res.frame_pair}")
        pose_result = run_pose_estimation_from_matches(
            match_result = res, 
            frame_shape = frame_shape, 
            fundamental_matrix = fundamental_data['fundamental_matrix'],
            visualise=PLOT_CAMERA_POSE_ESTIMATIONS
        )
        pose_estimations.append(pose_result)

    # 6. 3D Point Triangulation and Scene Visualisation
    print("\n3D Point Triangulation and Scene Visualisation:")
    reconstruct_scene(pose_estimations, ransac_results, Frames, detector_name=detector_type, use_bundle_adjustment=USE_BUNDLE_ADJUSTMENT, visualize=VISUALIZE_3D_RECONTRUCTION)


Running:ORB, func:detect_orb
[ORB] Matched 0 → 1: Before Lowe:3000 matches, After Lowe 1216 matches
[ORB] Matched 1 → 2: Before Lowe:3000 matches, After Lowe 1251 matches
[ORB] Matched 2 → 3: Before Lowe:3000 matches, After Lowe 1310 matches
[ORB] Matched 3 → 4: Before Lowe:3000 matches, After Lowe 1315 matches
[ORB] Matched 4 → 5: Before Lowe:3000 matches, After Lowe 1181 matches
[ORB] Matched 5 → 6: Before Lowe:3000 matches, After Lowe 942 matches
[ORB] Matched 6 → 7: Before Lowe:3000 matches, After Lowe 870 matches
[ORB] Matched 7 → 8: Before Lowe:3000 matches, After Lowe 762 matches
[ORB] Matched 8 → 9: Before Lowe:3000 matches, After Lowe 861 matches
[ORB] Matched 9 → 10: Before Lowe:3000 matches, After Lowe 1097 matches
Number of match results: 10

Fundamental Matrix and Epipolar Geometry:

Fundamental Matrix Computation for Frame Pair (0, 1)
Computing fundamental matrix...
Inlier matches: 692 out of 732 (94.54%)
Mean epipolar error: 1.0791 pixels
Max epipolar error: 2.9453 pixe

Generation of final analysis visualizations based on configured output flags. Creates side-by-side detector comparisons, performance metric plots, RANSAC filtering effectiveness and video montages of the feature detection process.

In [76]:
if PLOT_DETECTOR_BENCHMARK_COMPARISONS:
    compare_detectors_per_frame_cached(Frames, detected_features_all)
    plot_keypoint_counts_and_comp_time(results_benchmark_data)

if PLOT_RANSAC_FILTERING:
    plot_before_after_ransac_detected_features_all_algos(feature_matching_results_all_algos)

if CREATE_FEATURE_MONTAGE_VIDEOS: 
    create_feature_detection_montage(Frames, detected_features_all, ALGORITHM_NAME_FUNC_MAP)
